# Importing Libraries

In [2]:
import tensorflow as tf

2023-11-27 15:09:07.797558: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: SSE4.1 SSE4.2 AVX AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.


In [3]:
print(tf.__version__)

2.12.0


In [4]:
from tensorflow.keras import Model
from tensorflow.keras.layers import Conv3D, MaxPooling3D, UpSampling3D, Concatenate

In [5]:
import sys
sys.path.append('../../_EXPERIMENTS')

In [6]:
#import cactas as C
import os
import random
import numpy as np
import nrrd

In [7]:
gpu_available = tf.test.is_gpu_available()
gpu_available

Instructions for updating:
Use `tf.config.list_physical_devices('GPU')` instead.


False

In [8]:
import random
from tensorflow.keras.optimizers import Adam

# Defining Functions

In [9]:
def load_padded_images(IMG_DATAPATH):
    directory = IMG_DATAPATH
    min_filesize_bytes = 15728640 #15MB
    images = []
    for filename in os.listdir(directory):
        if (filename.startswith(".ipynb_checkpoints")):
            print()
        else:
            images.append(filename)
    return images

In [10]:
def load_padded_labels(LAB_DATAPATH):
    directory = LAB_DATAPATH
    min_filesize_bytes = 15728640 #15MB
    labels = []
    for filename in os.listdir(directory):
        if (filename.startswith(".ipynb_checkpoints")):
            print()
        else:
            labels.append(filename)
    return labels

In [11]:
def buildModel():
    inputs = tf.keras.Input(shape=(128, 128, 384, 1))

    conv1 = Conv3D(64, 3, activation='relu', padding='same')(inputs)
    pool1 = MaxPooling3D(pool_size=(2, 2, 2))(conv1)

    conv2 = Conv3D(128, 3, activation='relu', padding='same')(pool1) 
    pool2 = MaxPooling3D(pool_size=(2, 2, 2))(conv2)

    conv3 = Conv3D(256, 3, activation='relu', padding='same')(pool2)
    pool3 = MaxPooling3D(pool_size=(2, 2, 2))(conv3)

    conv4 = Conv3D(512, 3, activation='relu', padding='same')(pool3)
    pool4 = MaxPooling3D(pool_size=(2, 2, 2))(conv4)

    conv5 = Conv3D(1024, 3, activation='relu', padding='same')(pool4)

    up6 = Concatenate(axis=4)([UpSampling3D(size=(2, 2, 2))(conv5), conv4])
    conv6 = Conv3D(512, 3, activation='relu', padding='same')(up6)

    up7 = Concatenate(axis=4)([UpSampling3D(size=(2, 2, 2))(conv6), conv3])  
    conv7 = Conv3D(256, 3, activation='relu', padding='same')(up7)

    up8 = Concatenate(axis=4)([UpSampling3D(size=(2, 2, 2))(conv7), conv2])
    conv8 = Conv3D(128, 3, activation='relu', padding='same')(up8)

    up9 = Concatenate(axis=4)([UpSampling3D(size=(2, 2, 2))(conv8), conv1])
    conv9 = Conv3D(64, 3, activation='relu', padding='same')(up9)

    conv10 = Conv3D(1, 1, activation='sigmoid')(conv9)
    
    model = Model(inputs=inputs, outputs=conv10)
    
    return model

In [12]:
def split_data(images, labels, split_ratio=0.8):
    
    data = list(zip(images, labels))
    
    random.shuffle(data)

  # Get the split index
    split_index = int(len(data) * split_ratio)

  # Split data into train and test
    train_data = data[:split_index]
    test_data = data[split_index:]

  # Separate into images and labels
    X_train, y_train = zip(*train_data) 
    X_test, y_test = zip(*test_data)
   
    return X_train, X_test, y_train, y_test

In [13]:
def batch_normalize(data_path, file_list, batch_size):
    normalized_data = []
    for file in file_list:
        data, header = nrrd.read(data_path + "/" + file)
        normalized_data.append((data - np.min(data)) / (np.max(data) - np.min(data)))

        if len(normalized_data) == batch_size:
            yield normalized_data
            normalized_data = []

    if normalized_data:
        yield normalized_data

In [14]:
def normalization(IMG_DATAPATH, LAB_DATAPATH, X_train, y_train, X_test, y_test, batch_size):
    norm_X_train = []
    norm_y_train = []
    norm_X_test = []
    norm_y_test = []

    for batch in batch_normalize(IMG_DATAPATH, X_train, batch_size):
        norm_X_train.extend(batch)
    
    for batch in batch_normalize(LAB_DATAPATH, y_train, batch_size):
        norm_y_train.extend(batch)

    for batch in batch_normalize(IMG_DATAPATH, X_test, batch_size):
        norm_X_test.extend(batch)
    
    for batch in batch_normalize(LAB_DATAPATH, y_test, batch_size):
        norm_y_test.extend(batch)

    return norm_X_train, norm_y_train, norm_X_test, norm_y_test

# Loading Data and Normalizing

In [15]:
IMG_DATAPATH = os.path.join('ChunksESUS')
LAB_DATAPATH = os.path.join('Chunks_Labels_ESUS')

In [16]:
images = load_padded_images(IMG_DATAPATH)
labels = load_padded_labels(LAB_DATAPATH)

In [17]:
X_train, X_test, y_train, y_test = split_data(images, labels)

In [18]:
percentage_to_select = 30

num_samples_to_select = int(len(X_train) * percentage_to_select / 100)

random_indices = random.sample(range(len(X_train)), num_samples_to_select)

X_train_selected = [X_train[i] for i in random_indices]
y_train_selected = [y_train[i] for i in random_indices]

num_samples_to_select = int(len(X_test) * percentage_to_select / 100)
random_indices = random.sample(range(len(X_test)), num_samples_to_select)
X_test_selected = [X_test[i] for i in random_indices]
y_test_selected = [y_test[i] for i in random_indices]

In [19]:
len(X_train)

870

In [20]:
X_train, y_train, X_test, y_test = normalization(IMG_DATAPATH, LAB_DATAPATH, X_train_selected, y_train_selected, X_test_selected, y_test_selected, 2)

/tmp/ipykernel_1609211/1398699592.py:5: RuntimeWarning: invalid value encountered in divide
  normalized_data.append((data - np.min(data)) / (np.max(data) - np.min(data)))


In [21]:
len(X_train)

261

In [22]:
X_train = np.array(X_train)


In [23]:
X_test = np.array(X_test)

In [24]:
X_train = X_train.reshape(-1, 128, 128, 384, 1)
X_test = X_test.reshape(-1, 128, 128, 384, 1)

In [25]:
y_train = np.array(y_train)

In [26]:
y_test = np.array(y_test)

In [27]:
len(X_train)

261

In [28]:
X_train.shape

(261, 128, 128, 384, 1)

# Model Building

In [29]:
model = buildModel()

2023-11-27 15:19:14.845642: I tensorflow/core/common_runtime/process_util.cc:146] Creating new thread pool with default inter op setting: 2. Tune using inter_op_parallelism_threads for best performance.


In [30]:
model.compile(optimizer=Adam(lr=1e-4), loss='binary_crossentropy', metrics=['accuracy'])

In [ ]:
model.fit(X_train, y_train, batch_size=2, epochs=5, validation_data=(X_test, y_test))